In [ ]:
!pip install PyPDF2
!pip install transformers
!pip install torch
!pip install optimum
!pip install auto-gptq

import PyPDF2
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import files
import io

# Extract text from the PDF file
def extract_text_from_pdf(pdf_file):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Load the Qwen 2.5 model and tokenizer
def load_model_and_tokenizer():
    model_name = "Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4"  # GPTQ version for efficiency
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
        print("Successfully loaded Qwen 2.5 GPTQ model.")
    except Exception as e:
        print(f"Error loading Qwen 2.5 GPTQ model: {e}")
        print("Falling back to non-GPTQ version (standard Qwen 2.5).")
        model_name = "Qwen/Qwen2.5-7B"  # Non-GPTQ fallback
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer

# Improved chunking function to handle larger text
def chunk_text(text, chunk_size=1000):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(" ".join(current_chunk)) >= chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []  # Reset the current chunk

    # Append any remaining text in the last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# Generate complex questions from each chunk
def generate_complex_question(text_chunk, model, tokenizer):
    prompt = f"Based on the following text, generate a complex question with multiple-choice answers (a, b, c), the correct answer, and a helpful hint that guides the user towards the correct answer without giving it away directly:\n\n{text_chunk}\n\nQuestion, Answer, and Hint:"
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            num_beams=1,  # Use greedy search instead of beam search
            do_sample=False,  # Deterministic generation
            temperature=0.7,  # Lower temperature for more focused outputs
        )

    output = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    if "Question, Answer, and Hint:" in output:
        output = output.split("Question, Answer, and Hint:")[-1].strip()

    parts = output.split("\n")
    question = parts[0] if len(parts) > 0 else ""
    answer = parts[1].split("Answer: ")[-1].strip() if len(parts) > 1 else ""
    hint = parts[2].split("Hint: ")[-1].strip() if len(parts) > 2 else ""
    return question, answer, hint

# Ask questions with an option for hints
def ask_question_with_hint(question, hint):
    print(f"\nQuestion: {question}")
    hint_option = input("Do you need a hint? (yes/no): ").strip().lower()

    if hint_option == "yes":
        print(f"Hint: {hint}")

    user_answer = input("Your answer: ")
    return user_answer

# Function to calculate score based on user's answers and correct answers
def calculate_score(user_answers, correct_answers):
    score = sum(1 for user, correct in zip(user_answers, correct_answers)
                if user.lower().strip() == correct.lower().strip())
    return score

def main():
    print("Please upload your PDF file.")
    uploaded = files.upload()

    if not uploaded:
        print("No file was uploaded. Exiting.")
        return

    for filename, content in uploaded.items():
        print(f"Processing file: {filename}")

        try:
            pdf_file = io.BytesIO(content)
            model, tokenizer = load_model_and_tokenizer()
            text = extract_text_from_pdf(pdf_file)

            chunks = chunk_text(text)
            all_questions = []
            correct_answers = []
            hints = []
            paragraphs = []

            # Generate complex questions, correct answers, and hints for each paragraph
            question_count = 0
            for chunk in chunks:
                if question_count >= 5:
                    break
                question, answer, hint = generate_complex_question(chunk, model, tokenizer)
                all_questions.append(question)
                correct_answers.append(answer)
                hints.append(hint)
                paragraphs.append(chunk)
                question_count += 1

            # Ask each question and get the user's answers
            user_answers = []
            for i, (question, paragraph, hint) in enumerate(zip(all_questions, paragraphs, hints), 1):
                print(f"\nParagraph {i}:")
                print(paragraph)
                user_answer = ask_question_with_hint(question, hint)
                user_answers.append(user_answer)

            # Calculate the user's score based on their answers
            score = calculate_score(user_answers, correct_answers)

            # Display the final score and correct answers
            print(f"\nYour final score is: {score} out of {len(correct_answers)}")
            print("\nCorrect Answers:")
            for i, (question, correct_answer, hint) in enumerate(zip(all_questions, correct_answers, hints), 1):
                print(f"Q{i}: {question}\nCorrect Answer: {correct_answer}\nHint: {hint}\n")

        except Exception as e:
            print(f"An error occurred while processing the file: {str(e)}")

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.7/453.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/

Saving Deep Learning VS Machine Learning The Ultimate Comparison by Careervira Medium.pdf to Deep Learning VS Machine Learning The Ultimate Comparison by Careervira Medium.pdf
Processing file: Deep Learning VS Machine Learning The Ultimate Comparison by Careervira Medium.pdf


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
